# Time series
Following https://www.tutorialspoint.com/time_series

In [ ]:
!pip install hurst

In [ ]:
import numpy
import pandas
import datetime
import matplotlib.pyplot as plt
from sklearn import metrics

from math import sqrt

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import hurst
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from scipy.stats.stats import pearsonr
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
df = pandas.read_csv("data/AirQualityUCI.csv", sep = ";", decimal = ",")
df = df.iloc[ : , 0:14]
df = df[df['Date'].notnull()]

df['DateTime'] = (df.Date) + ' ' + (df.Time)
df.DateTime = df.DateTime.apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y %H.%M.%S'))
df.index = df.DateTime

df.head()

In [ ]:
plt.boxplot(df[['T','C6H6(GT)']].values)

## Statistics

In [ ]:
print (f"Mean: {numpy.mean(df['T'])}",
       f"Standard deviation: {numpy.std(df['T'])}",
       f"Maximum Temperature: {max(df['T'])}",
       f"Minimum Temperature: {min(df['T'])}",
       sep="\n"
)

## Showing 1st naïve method

In [ ]:
df['T']
df['T_t-1'] = df['T'].shift(1)
df_naive = df[['T','T_t-1']][1:]
true = df_naive['T']
prediction = df_naive['T_t-1']
error = sqrt(metrics.mean_squared_error(true,prediction))
print ('RMSE for Naive Method 1: ', error)

## Showing 2nd naive method

In [ ]:
df['T_rm'] = df['T'].rolling(3).mean().shift(1)
df_naive = df[['T','T_rm']].dropna()
true = df_naive['T']
prediction = df_naive['T_rm']
error = sqrt(metrics.mean_squared_error(true,prediction))
print ('RMSE for Naive Method 2: ', error)

## Showing ACP

In [ ]:
split = len(df) - int(0.2*len(df))
train, test = df['T'][0:split], df['T'][split:]
plot_acf(train, lags = 100)
plt.show()

## Showing PACP

In [ ]:
plot_pacf(train, lags = 100)
plt.show()

# ARIMA

## Check stationarity

In [ ]:
result = adfuller(train)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
H, c,data = hurst.compute_Hc(train)
print("H = {:.4f}, c = {:.4f}".format(H,c))

In [ ]:
model = ARIMA(train.values, order=(5, 0, 2))
model_fit = model.fit(disp=False)

predictions = model_fit.predict(len(test))
test_ = pandas.DataFrame(test)
test_['predictions'] = predictions[0:1871]

plt.plot(df['T'])
plt.plot(test_.predictions)
plt.show()

error = sqrt(metrics.mean_squared_error(test.values,predictions[0:1871]))
print ('Test RMSE for ARIMA: ', error)

## VARMA

In [ ]:
df_multi = df[['T', 'C6H6(GT)']]
split = len(df) - int(0.2*len(df))
train_multi, test_multi = df_multi[0:split], df_multi[split:]

model = VARMAX(train_multi, order = (2,1))
model_fit = model.fit()
predictions_multi = model_fit.forecast( steps=len(test_multi))

plt.plot(train_multi['T'])
plt.plot(test_multi['T'])
plt.plot(predictions_multi.iloc[:,0:1], '--')
plt.show()

plt.plot(train_multi['C6H6(GT)'])
plt.plot(test_multi['C6H6(GT)'])
plt.plot(predictions_multi.iloc[:,1:2], '--')
plt.show()

## SARIMA with Exogenous Variables (SARIMAX)

In [ ]:
x = train_multi['T'].values
y = train_multi['C6H6(GT)'].values

corr , p = pearsonr(x,y)
print ('Corelation Coefficient =', corr,'\nP-Value =',p)

model = SARIMAX(x, exog = y, order = (2, 0, 2), seasonal_order = (2, 0, 1, 1), enforce_stationarity=False, enforce_invertibility = False)
model_fit = model.fit(disp = False)

y_ = test_multi['C6H6(GT)'].values
predicted = model_fit.predict(exog=y_)
test_multi_ = pandas.DataFrame(test)
test_multi_['predictions'] = predicted[0:1871]

plt.plot(train_multi['T'])
plt.plot(test_multi_['T'])
plt.plot(test_multi_.predictions, '--')

# Exponential Smoothing

## Triple Exponential Smoothing

In [ ]:
model = ExponentialSmoothing(train.values, trend= )
model_fit = model.fit()

predictions_ = model_fit.predict(len(test))

plt.plot(test.values)
plt.plot(predictions_[1:1871])